In [35]:
import pickle
X_hr,y_stress,y_participant,X_time = pickle.load(open('../data/tabular_data_60_seconds_ppg_rr.p','rb'))

In [36]:
X_hr = X_hr.reshape(-1,30,1)

In [37]:
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,LeaveOneGroupOut,LeavePGroupsOut
from sklearn.metrics import accuracy_score
from tensorflow import keras
from tensorflow.keras import backend as K 
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import numpy as np
%matplotlib notebook
from keras.layers import Conv1D,Reshape,BatchNormalization,TimeDistributed, \
Dropout,Input,MaxPooling1D,Flatten,Dense,Input, GaussianNoise,LSTM, Bidirectional, Input
from keras.models import Model, Sequential
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer,LabelEncoder
from sklearn.metrics.pairwise import euclidean_distances

In [38]:
train_participant = y_participant.copy()

In [39]:
np.unique(y_participant),train_participant.shape

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]), (42928,))

In [40]:
train_x = X_hr
train_y = y_participant

In [24]:
train_x, test_x, train_y, test_y,participant_ids_train, participant_ids_test = train_test_split(X_hr,
                                                            train_participant,
                                                            y_participant,
                                                            test_size = 0.2,
                                                            random_state=41,
                                                            stratify=y_participant)
train_x, val_x, train_y, val_y, participant_ids_train, participant_ids_val  = train_test_split(train_x,
                                                            train_y,
                                                            participant_ids_train,
                                                            test_size = 0.1,
                                                            random_state=41,
                                                            stratify=participant_ids_train)
train_x.shape,test_x.shape,val_x.shape,train_y.shape,test_y.shape,val_y.shape,participant_ids_train.shape

((30907, 30, 1),
 (8586, 30, 1),
 (3435, 30, 1),
 (30907,),
 (8586,),
 (3435,),
 (30907,))

In [41]:
def get_model(input_shape=(30,1),act='relu',loss="categorical_crossentropy",opt='adam',n_classes=350):
    model =  Sequential()
    model.add(Conv1D(100,10,input_shape=input_shape,activation='linear',kernel_initializer='normal',padding='same'))
    model.add(Conv1D(100,10,input_shape=input_shape,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
    model.add(BatchNormalization())
    model.add(Dropout(.2))
    model.add(Conv1D(200,10,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
    model.add(BatchNormalization())
    model.add(Dropout(.2))
    model.add(Conv1D(300,10,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
    model.add(BatchNormalization())
    model.add(Conv1D(50,2,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
    model.add(BatchNormalization())
    model.add(Dropout(.2))
    model.add(Flatten())
    model.add(Dense(n_classes//2,activation=None))
    model.add(Dense(n_classes,activation=None))
#     model.add(Dense(n_classes,activation='softmax',kernel_initializer='normal'))
    model.add(tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)))
#     model.compile(loss=loss,optimizer=opt,metrics=['acc'])

    return model

In [42]:
n_classes = 1000
model = get_model(input_shape=(30,1),n_classes=n_classes)

In [43]:
model.compile(optimizer='adam',loss=tfa.losses.TripletSemiHardLoss())

In [44]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_5 (Conv1D)            (None, 30, 100)           1100      
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 30, 100)           100100    
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 15, 100)           0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 15, 100)           400       
_________________________________________________________________
dropout_3 (Dropout)          (None, 15, 100)           0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 15, 200)           200200    
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 7, 200)           

In [45]:
from keras.models import load_model
filepath = '../model_files/base_cnn_60_seconds_ppg_hr_wesad.hdf5'

In [46]:
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=40)
callbacks_list = [es,checkpoint]
history = model.fit(train_x,train_y,validation_data=(val_x,val_y), epochs=400, batch_size=200,
          callbacks=callbacks_list,shuffle=True)

Epoch 1/400
215/215 [==============================] - ETA: 0s - loss: 0.9885
Epoch 00001: val_loss improved from inf to 0.98806, saving model to ../model_files/base_cnn_60_seconds_ppg_hr_wesad.hdf5
215/215 [==============================] - 3s 15ms/step - loss: 0.9885 - val_loss: 0.9881
Epoch 2/400
212/215 [============================>.] - ETA: 0s - loss: 0.9874
Epoch 00002: val_loss improved from 0.98806 to 0.98795, saving model to ../model_files/base_cnn_60_seconds_ppg_hr_wesad.hdf5
215/215 [==============================] - 3s 13ms/step - loss: 0.9874 - val_loss: 0.9880
Epoch 3/400
212/215 [============================>.] - ETA: 0s - loss: 0.9873
Epoch 00003: val_loss improved from 0.98795 to 0.98632, saving model to ../model_files/base_cnn_60_seconds_ppg_hr_wesad.hdf5
215/215 [==============================] - 3s 14ms/step - loss: 0.9872 - val_loss: 0.9863
Epoch 4/400
211/215 [============================>.] - ETA: 0s - loss: 0.9872
Epoch 00004: val_loss did not improve from 0.98

215/215 [==============================] - 3s 14ms/step - loss: 0.9285 - val_loss: 0.9050
Epoch 33/400
213/215 [============================>.] - ETA: 0s - loss: 0.9263
Epoch 00033: val_loss did not improve from 0.90502
215/215 [==============================] - 3s 13ms/step - loss: 0.9264 - val_loss: 0.9112
Epoch 34/400
211/215 [============================>.] - ETA: 0s - loss: 0.9239
Epoch 00034: val_loss improved from 0.90502 to 0.89778, saving model to ../model_files/base_cnn_60_seconds_ppg_hr_wesad.hdf5
215/215 [==============================] - 3s 14ms/step - loss: 0.9241 - val_loss: 0.8978
Epoch 35/400
211/215 [============================>.] - ETA: 0s - loss: 0.9202
Epoch 00035: val_loss did not improve from 0.89778
215/215 [==============================] - 3s 13ms/step - loss: 0.9203 - val_loss: 0.8983
Epoch 36/400
212/215 [============================>.] - ETA: 0s - loss: 0.9219
Epoch 00036: val_loss improved from 0.89778 to 0.88976, saving model to ../model_files/base_cnn_6

Epoch 65/400
213/215 [============================>.] - ETA: 0s - loss: 0.7569
Epoch 00065: val_loss did not improve from 0.67854
215/215 [==============================] - 3s 13ms/step - loss: 0.7572 - val_loss: 0.6963
Epoch 66/400
211/215 [============================>.] - ETA: 0s - loss: 0.7662
Epoch 00066: val_loss did not improve from 0.67854
215/215 [==============================] - 3s 13ms/step - loss: 0.7659 - val_loss: 0.8530
Epoch 67/400
211/215 [============================>.] - ETA: 0s - loss: 0.7622
Epoch 00067: val_loss improved from 0.67854 to 0.67310, saving model to ../model_files/base_cnn_60_seconds_ppg_hr_wesad.hdf5
215/215 [==============================] - 3s 14ms/step - loss: 0.7621 - val_loss: 0.6731
Epoch 68/400
215/215 [==============================] - ETA: 0s - loss: 0.7488
Epoch 00068: val_loss did not improve from 0.67310
215/215 [==============================] - 3s 13ms/step - loss: 0.7488 - val_loss: 0.7050
Epoch 69/400
211/215 [========================

Epoch 98/400
213/215 [============================>.] - ETA: 0s - loss: 0.6644
Epoch 00098: val_loss did not improve from 0.57980
215/215 [==============================] - 3s 13ms/step - loss: 0.6642 - val_loss: 0.5950
Epoch 99/400
212/215 [============================>.] - ETA: 0s - loss: 0.6704
Epoch 00099: val_loss improved from 0.57980 to 0.56517, saving model to ../model_files/base_cnn_60_seconds_ppg_hr_wesad.hdf5
215/215 [==============================] - 3s 14ms/step - loss: 0.6706 - val_loss: 0.5652
Epoch 100/400
212/215 [============================>.] - ETA: 0s - loss: 0.6570
Epoch 00100: val_loss did not improve from 0.56517
215/215 [==============================] - 3s 13ms/step - loss: 0.6571 - val_loss: 0.6021
Epoch 101/400
213/215 [============================>.] - ETA: 0s - loss: 0.6613
Epoch 00101: val_loss did not improve from 0.56517
215/215 [==============================] - 3s 13ms/step - loss: 0.6620 - val_loss: 0.6596
Epoch 102/400
213/215 [=====================

211/215 [============================>.] - ETA: 0s - loss: 0.5738
Epoch 00131: val_loss did not improve from 0.42801
215/215 [==============================] - 3s 13ms/step - loss: 0.5735 - val_loss: 0.4368
Epoch 132/400
213/215 [============================>.] - ETA: 0s - loss: 0.5646
Epoch 00132: val_loss improved from 0.42801 to 0.41346, saving model to ../model_files/base_cnn_60_seconds_ppg_hr_wesad.hdf5
215/215 [==============================] - 3s 14ms/step - loss: 0.5635 - val_loss: 0.4135
Epoch 133/400
211/215 [============================>.] - ETA: 0s - loss: 0.5558
Epoch 00133: val_loss did not improve from 0.41346
215/215 [==============================] - 3s 13ms/step - loss: 0.5557 - val_loss: 0.4144
Epoch 134/400
214/215 [============================>.] - ETA: 0s - loss: 0.5601
Epoch 00134: val_loss improved from 0.41346 to 0.40843, saving model to ../model_files/base_cnn_60_seconds_ppg_hr_wesad.hdf5
215/215 [==============================] - 3s 14ms/step - loss: 0.5602 -

215/215 [==============================] - 3s 14ms/step - loss: 0.4277 - val_loss: 0.2413
Epoch 163/400
213/215 [============================>.] - ETA: 0s - loss: 0.4167
Epoch 00163: val_loss did not improve from 0.24134
215/215 [==============================] - 3s 13ms/step - loss: 0.4161 - val_loss: 0.2664
Epoch 164/400
211/215 [============================>.] - ETA: 0s - loss: 0.4247
Epoch 00164: val_loss did not improve from 0.24134
215/215 [==============================] - 3s 13ms/step - loss: 0.4251 - val_loss: 0.2779
Epoch 165/400
212/215 [============================>.] - ETA: 0s - loss: 0.4108
Epoch 00165: val_loss did not improve from 0.24134
215/215 [==============================] - 3s 13ms/step - loss: 0.4109 - val_loss: 0.3128
Epoch 166/400
212/215 [============================>.] - ETA: 0s - loss: 0.4243
Epoch 00166: val_loss did not improve from 0.24134
215/215 [==============================] - 3s 13ms/step - loss: 0.4241 - val_loss: 0.2960
Epoch 167/400
214/215 [===

Epoch 198/400
212/215 [============================>.] - ETA: 0s - loss: 0.3321
Epoch 00198: val_loss did not improve from 0.19484
215/215 [==============================] - 3s 13ms/step - loss: 0.3323 - val_loss: 0.2159
Epoch 199/400
213/215 [============================>.] - ETA: 0s - loss: 0.3377
Epoch 00199: val_loss did not improve from 0.19484
215/215 [==============================] - 3s 13ms/step - loss: 0.3377 - val_loss: 0.1970
Epoch 200/400
211/215 [============================>.] - ETA: 0s - loss: 0.3435
Epoch 00200: val_loss improved from 0.19484 to 0.19011, saving model to ../model_files/base_cnn_60_seconds_ppg_hr_wesad.hdf5
215/215 [==============================] - 3s 14ms/step - loss: 0.3429 - val_loss: 0.1901
Epoch 201/400
211/215 [============================>.] - ETA: 0s - loss: 0.3342
Epoch 00201: val_loss improved from 0.19011 to 0.18024, saving model to ../model_files/base_cnn_60_seconds_ppg_hr_wesad.hdf5
215/215 [==============================] - 3s 14ms/step - 

211/215 [============================>.] - ETA: 0s - loss: 0.2783
Epoch 00232: val_loss improved from 0.16013 to 0.15755, saving model to ../model_files/base_cnn_60_seconds_ppg_hr_wesad.hdf5
215/215 [==============================] - 3s 14ms/step - loss: 0.2788 - val_loss: 0.1576
Epoch 233/400
211/215 [============================>.] - ETA: 0s - loss: 0.2803
Epoch 00233: val_loss did not improve from 0.15755
215/215 [==============================] - 3s 13ms/step - loss: 0.2803 - val_loss: 0.1644
Epoch 234/400
213/215 [============================>.] - ETA: 0s - loss: 0.2774
Epoch 00234: val_loss did not improve from 0.15755
215/215 [==============================] - 3s 13ms/step - loss: 0.2770 - val_loss: 0.1620
Epoch 235/400
214/215 [============================>.] - ETA: 0s - loss: 0.2766
Epoch 00235: val_loss did not improve from 0.15755
215/215 [==============================] - 3s 13ms/step - loss: 0.2767 - val_loss: 0.1654
Epoch 236/400
212/215 [============================>.] -

Epoch 269/400
212/215 [============================>.] - ETA: 0s - loss: 0.2568
Epoch 00269: val_loss did not improve from 0.15439
215/215 [==============================] - 3s 13ms/step - loss: 0.2568 - val_loss: 0.1549
Epoch 270/400
211/215 [============================>.] - ETA: 0s - loss: 0.2574
Epoch 00270: val_loss did not improve from 0.15439
215/215 [==============================] - 3s 13ms/step - loss: 0.2569 - val_loss: 0.1568
Epoch 271/400
213/215 [============================>.] - ETA: 0s - loss: 0.2615
Epoch 00271: val_loss did not improve from 0.15439
215/215 [==============================] - 3s 13ms/step - loss: 0.2614 - val_loss: 0.1571
Epoch 272/400
212/215 [============================>.] - ETA: 0s - loss: 0.2635
Epoch 00272: val_loss did not improve from 0.15439
215/215 [==============================] - 3s 13ms/step - loss: 0.2625 - val_loss: 0.1574
Epoch 273/400
212/215 [============================>.] - ETA: 0s - loss: 0.2532
Epoch 00273: val_loss did not improv

213/215 [============================>.] - ETA: 0s - loss: 0.2514
Epoch 00305: val_loss did not improve from 0.15354
215/215 [==============================] - 3s 13ms/step - loss: 0.2510 - val_loss: 0.1550
Epoch 306/400
215/215 [==============================] - ETA: 0s - loss: 0.2363
Epoch 00306: val_loss did not improve from 0.15354
215/215 [==============================] - 3s 13ms/step - loss: 0.2363 - val_loss: 0.1576
Epoch 307/400
214/215 [============================>.] - ETA: 0s - loss: 0.2370
Epoch 00307: val_loss did not improve from 0.15354
215/215 [==============================] - 3s 13ms/step - loss: 0.2370 - val_loss: 0.2316
Epoch 308/400
214/215 [============================>.] - ETA: 0s - loss: 0.2393
Epoch 00308: val_loss did not improve from 0.15354
215/215 [==============================] - 3s 13ms/step - loss: 0.2392 - val_loss: 0.1570
Epoch 309/400
214/215 [============================>.] - ETA: 0s - loss: 0.2412
Epoch 00309: val_loss did not improve from 0.15354

In [47]:
model = load_model(filepath)
train_embeddings = model.predict(train_x)

In [48]:
def give_rank_accuracy(train_embeddings,train_y,rank = 10):
    distance_matrix = euclidean_distances(train_embeddings)
    distance_matrix_2d = np.zeros((distance_matrix.shape[0],
                               distance_matrix.shape[0],
                               2))
    distance_matrix_2d[:,:,0] = distance_matrix
    train_y_matrix = np.concatenate([train_y.reshape(1,-1)]*distance_matrix.shape[0])
    distance_matrix_2d[:,:,1] = train_y_matrix
    match = 0
    for i in range(distance_matrix.shape[0]):
        row = distance_matrix_2d[i]
        row = row[row[:,0].argsort(),:]
        if rank==1:
            if train_y[i]==row[1,1]:
                match+=1
        else:
            row = set(row[1:(rank+1)][:,1])
            if train_y[i] in row:
                match+=1
    return match/distance_matrix.shape[0]

In [49]:
rank_1 = give_rank_accuracy(train_embeddings,train_y,rank = 1)
rank_5 = give_rank_accuracy(train_embeddings,train_y,rank = 5)
rank_10 = give_rank_accuracy(train_embeddings,train_y,rank = 10)
print(rank_1,rank_5,rank_10)

0.8870667163622811 0.9764256429370108 0.9944325382035035


In [30]:
test_embeddings = model.predict(test_x)

In [31]:
rank_1 = give_rank_accuracy(test_embeddings,test_y,rank = 1)
rank_5 = give_rank_accuracy(test_embeddings,test_y,rank = 5)
rank_10 = give_rank_accuracy(test_embeddings,test_y,rank = 10)
print(rank_1,rank_5,rank_10)

0.8382250174703005 0.9588865595154903 0.9765897973445143


In [50]:
test_x,y_stress,test_y,X_time = pickle.load(open('../data/tabular_data_60_seconds_ppg_rr_dalia.p','rb'))

In [51]:
test_x = test_x.reshape(-1,30,1)

In [52]:
test_embeddings = model.predict(test_x)
rank_1 = give_rank_accuracy(test_embeddings,test_y,rank = 1)
rank_5 = give_rank_accuracy(test_embeddings,test_y,rank = 5)
rank_10 = give_rank_accuracy(test_embeddings,test_y,rank = 10)
print(rank_1,rank_5,rank_10)

0.8839471855455178 0.9749826268241835 0.995135510771369


In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
print(classification_report(train_y,train_y_pred))
print(confusion_matrix(train_y,train_y_pred))